## В этом задании вам нужно реализовать распределенное обучение линейной регрессии методом градиентного спуска c помощью RDD API

In [1]:
import os
import sys

SPARK_HOME = "/usr/hdp/current/spark2-client"
PYSPARK_PYTHON = "/opt/conda/envs/dsenv/bin/python"
os.environ["PYSPARK_PYTHON"]= PYSPARK_PYTHON
os.environ["SPARK_HOME"] = SPARK_HOME

PYSPARK_HOME = os.path.join(SPARK_HOME, "python/lib")
sys.path.insert(0, os.path.join(PYSPARK_HOME, "py4j-0.10.7-src.zip"))
sys.path.insert(0, os.path.join(PYSPARK_HOME, "pyspark.zip"))

In [2]:
import random
SPARK_UI_PORT = random.choice(range(10000, 10200))
print(f"Spark UI port: {SPARK_UI_PORT}")

Spark UI port: 10186


In [3]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf()
conf.set("spark.ui.port", SPARK_UI_PORT)

spark = SparkSession.builder.config(conf=conf).appName("Gradient descent").getOrCreate()

In [4]:
sc = spark.sparkContext

In [5]:
from pyspark.mllib.regression import LabeledPoint, LinearRegressionWithSGD
from numpy import array
import random

### Шаг 1
Реализуйте функцию `gen_point`, которая генерирует точки для функции `y = 10 * x1 + 25 * x2`. `x1` должен генерироваться из равномерного распределения `[-100, 100]`, `x2` из равномерного распределения `[-20, 60]`. Так же к точке необходимо добавить равномерный шум `[-2, 2]`. Функция должна возвращать объект `LabeledPoint`

In [8]:
# Ваш код здесь
import random

def gen_point():
    x1 = random.uniform(-100, 100)
    x2 = random.uniform(-20, 60)
    return LabeledPoint(x1 * 10.0 + x2 * 25.0 + random.uniform(-2, 2), [x1, x2])

### Шаг 2
Сгенерируйте RDD, состоящую из 100000 точек

In [11]:
# Ваш код здесь
collection = [gen_point() for i in range (100000)]
rdd = sc.parallelize(collection, numSlices=4)
rdd.take(5)

[LabeledPoint(-485.43084623376404, [-44.563840327774365,-1.5607763144258087]),
 LabeledPoint(605.9549673920567, [-15.755262688576295,30.58940474571196]),
 LabeledPoint(-167.84954364219197, [-49.57641336180605,13.149207990705989]),
 LabeledPoint(359.3835062105808, [-96.34943771764364,52.880540316582554]),
 LabeledPoint(403.3508537655214, [46.39792003833122,-2.440356627675115])]

### Шаг 3
Используйте `LinearRegressionWithSGD` для обучения модели. 100 итераций, шаг обучения - 0.0001. Выведите веса модели

In [17]:
# Ваш код здесь
#training = spark.createDataFrame(rdd, schema = ["label", "features"])

sgd = LinearRegressionWithSGD().train(rdd, step=0.0001)
# featuresCol="features", labelCol="label", maxIter=100, regParam=0.01

In [19]:
sgd

(weights=[9.97696253142852,20.673355580906133], intercept=0.0)

## Реализуйте градиентный спуск своими силами

### Шаг 4
Разделите RDD с точками на два: `x`, содержащий факторы и `y`, содержащий ответы

In [24]:
# Ваш код здесь
x = rdd.map(lambda x: x.features).cache()
y = rdd.map(lambda x: x.label).cache()
x.take(1), y.take(1)

([DenseVector([-44.5638, -1.5608])], [-485.43084623376404])

### Шаг 5
Проиницилизируйте начальные веса модели (не используйте bias) и шаг обучения, как на шаге 3 

In [25]:
# Ваш код здесь
weights = array([0, 0])
step=0.0001

### Шаг 6
Создайте RDD предсказаний линейной модели с текущими весами

In [29]:
# Ваш код здесь
import numpy as np
ys = x.map(lambda x: np.dot(weights, x))
ys.take(5)

[0.0, 0.0, 0.0, 0.0, 0.0]

### Шаг 7
Создайте RDD, вычисляющую градиента функционала потерь линейной регресси $(X_i \cdot weights - y_i) * X_i$

In [30]:
# Ваш код здесь
grad = x.zip(y).map(lambda xy: (xy[0].dot(weights) - xy[1]) * xy[0] )
grad.take(5)

[DenseVector([-21632.6627, -757.649]),
 DenseVector([9546.9797, -18535.8018]),
 DenseVector([-8321.3784, 2207.0886]),
 DenseVector([34626.3987, -19004.394]),
 DenseVector([-18714.6407, 984.3199])]

### Шаг 8
Усредните градиенты по датасету и сделайте шаг обновления градиента

In [33]:
# Ваш код здесь
weights = weights - step * grad.mean()
weights

DenseVector([3.3001, 2.3245])

### Шаг 9
Теперь соединим все вместе. На каждой итерации выводите текущие веса и среднеквадратическую ошибку

In [38]:
# Ваш код здесь
for i in range(10):
    ys = x.map(lambda x: np.dot(weights, x))
    grad = x.zip(y).map(lambda xy: (xy[0].dot(weights) - xy[1]) * xy[0] )
    weights = weights - step * grad.mean()
    mse = y.zip(ys).map(lambda x: (x[0] - x[1])**2).mean()
    print(f"Step no {i}, weights: {weights}, MSE: {mse}")

Step no 0, weights: [9.999335071583767,24.58796413024256], MSE: 159.3152248617526
Step no 1, weights: [9.999403984108387,24.626326119249775], MSE: 131.25314135601394
Step no 2, weights: [9.999466441019093,24.661114646422696], MSE: 108.17558803369859
Step no 3, weights: [9.999523055693231,24.69266258368005], MSE: 89.19718725896519
Step no 4, weights: [9.999574380388653,24.72127179555861], MSE: 73.58982668064037
Step no 5, weights: [9.99962091325563,24.74721602758294], MSE: 60.75472497141689
Step no 6, weights: [9.99966310425906,24.770743525581018], MSE: 50.19945939275474
Step no 7, weights: [9.999701360257003,24.79207941100827], MSE: 41.51907384068236
Step no 8, weights: [9.999736049410801,24.811427835007848], MSE: 34.38054257672577
Step no 9, weights: [9.999767505053734,24.82897393181804], MSE: 28.509993590072174


In [39]:
spark.stop()